## Import librairies

In [16]:
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
import keras_ocr
import math

## Rognage

In [192]:
image = cv2.imread('leo0.png')

In [193]:
image.shape

(1536, 2561, 3)

In [17]:
    try:
        os.mkdir('preprocessed_1')
    except:
        pass

In [18]:
def crop_img(number_picture:int, constellation_name:str):
    try:
        os.mkdir(f'preprocessed_1/{constellation_name}')
    except:
        pass
    y1=70
    y2=75
    x=0
    w=2561
    h=1536
    for x in range(0,number_picture,1):
        image = cv2.imread(f'./constellations/{constellation_name}/{constellation_name}{x}.png')
        crop = image[y1:-y2+h, x:x+w]
        cv2.imwrite(f'./preprocessed_1/{constellation_name}/{constellation_name}{x}.png',crop)

In [196]:
crop_img(46, "leo")

In [19]:
crop_img(46, "orion")

### Informations sur les coordonnées choisis pour rogner les images   
   
y1 représente la valeur comprise entre la valeur maximale de l'axe x,y et la nouvelle valeur maximale de l'axe x,y.    
y1 permet de rogner la hauteur haute de l'image.
   
y2 représente la valeur qui se situe entre l'origine de l'axe x,y et la valeur de la nouvelle origine de l'axe x,y.   
y2 permet de rogner la hauteur basse de l'image.   
   

Pour connaitre les nouvelles valeurs souhaitées (y1 et y2), il a fallu ouvrir une image du dataset sous le logiciel Paint.  
Une fois ouverte, il a fallu activer le quadrillage et utiliser l'échelle pour connaître le nombre de pixels compris entre les valeurs d'origines et les valeurs souhaitées.  
Les images étant toutes de la même dimension, il a ensuite fallu généraliser un unique rognage pour toutes les images.

## Suppression du texte

In [23]:
    try:
        os.mkdir('preprocessed_1')
    except:
        pass

In [24]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

pipeline = keras_ocr.pipeline.Pipeline()

def inpaint_text(pipeline, constellation_name:str, number_picture:int):
    try:
        os.mkdir(f'preprocessed_1/{constellation_name}')
    except:
        pass
    # read image
    for x in range(0,number_picture,1):
        img = keras_ocr.tools.read(f'./preprocessed_1/{constellation_name}/{constellation_name}{x}.png')
        # generate (word, box) tuples 
        prediction_groups = pipeline.recognize([img])
        mask = np.zeros(img.shape[:2], dtype="uint8")
        for box in prediction_groups[0]:
            x0, y0 = box[1][0]
            x1, y1 = box[1][1] 
            x2, y2 = box[1][2]
            x3, y3 = box[1][3] 
            
            x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
            x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
            
            thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
            
            cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255, thickness)
            img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
            img_resize = cv2.resize(img, dsize=(780,423))
            cv2.imwrite(f'./preprocessed_1/{constellation_name}/{constellation_name}{x}.png',img_resize)

Looking for C:\Users\vosco\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\vosco\.keras-ocr\crnn_kurapan.h5


In [25]:
inpaint_text(pipeline, 'leo', 46)

1/1 [==============================] - 0s 444ms/step


In [26]:
inpaint_text(pipeline, 'orion', 46)

1/1 [==============================] - 1s 900ms/step


J'ai essayé d'utiliser tesseract mais la librairie est compliquée à installer (setting de path non mentionné, fichiers manquants) et par manque de temps, je vais utiliser keras-ocr qui ne pose pas de problème lors de l'installation car appartenant au framework keras et qui possède un tuto permettant de supprimer des mots sur une image.